In [159]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [160]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns",100, "display.width",2000, "display.max_colwidth",100)

from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.model_selection import train_test_split
import re

#### Read Data

In [161]:
input_df = pd.read_csv("protac.csv")
input_df = input_df.rename(columns={"E3 ligase":"E3ligase"})
print("input_df: {:,} x {:,}".format(*input_df.shape), "(input df) \n")

print("Counts of distinct values in columns")
for x in ["Target","E3ligase"]:
  tmp = input_df.groupby(x, as_index=False).size().sort_values("size", ascending=False, ignore_index=True)
  print(f"col:{x}  (#unique:{tmp.shape[0]:,})")
  print(tmp.head(20).T, "\n")

del tmp

input_df: 3,939 x 84 (input df) 

Counts of distinct values in columns
col:Target  (#unique:289)
         0     1     2    3     4    5    6        7     8     9       10    11    12     13     14    15          16                17   18    19
Target   ER  BRD4  CDK4   AR  CDK6  BTK  ALK  BCR-ABL  BRD3  BRD2  BCL-xL  MEK1  CDK2  PARP1  HDAC6  BRD9  BRAF V600E  EGFR L858R/T790M  FAK  CDK9
size    182   164   143  141   139  106   97       96    77    77      77    76    70     63     60    53          53                52   52    52 

col:E3ligase  (#unique:12)
            0     1      2     3     4    5    6       7       8       9       10    11
E3ligase  CRBN   VHL  cIAP1  XIAP  MDM2  AhR  IAP  DCAF15  DCAF16  RNF114  DCAF11  RNF4
size      2571  1152    122    34    30    9    5       4       4       4       3     1 



In [207]:
print('Number of Rows with explicit DC50: ' + str((input_df.shape[0] - input_df['DC50 (nM)'].isna().sum())))
print('Number of Rows with percent degradation: ' + str((input_df.shape[0] - input_df["Percent degradation (%)"].isna().sum())))
#input_df = input_df.dropna(subset=['DC50 (nM)',"Percent degradation (%)"],how='all')
input_df = input_df.dropna(subset=['DC50 (nM)'])
print("input_df: {:,} x {:,}".format(*input_df.shape), "(input df) \n")

Number of Rows with explicit DC50: 638
Number of Rows with percent degradation: 207
input_df: 638 x 84 (input df) 



#### Features

Target and E3Ligase

In [208]:
model_df = pd.concat([
    pd.get_dummies(input_df["Target"], prefix="tgt"),
    pd.get_dummies(input_df["E3ligase"], prefix="e3")
  ], axis=1).reset_index(drop=True)

print("model_df: {:,} x {:,}".format(*model_df.shape), "(target and E3ligase OHE features)")

model_df: 638 x 91 (target and E3ligase OHE features)


Molecular Fingerprint

In [209]:
bits = 256

def get_morgan_fp(smiles:str, nbits:int=bits):
  m = Chem.MolFromSmiles(smiles)
  fingerprint = AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=nbits)
  array = np.zeros((0,), dtype=np.int8)
  DataStructs.ConvertToNumpyArray(fingerprint, array)
  return array

smiles = [get_morgan_fp(x).tolist() for x in input_df["Smiles"]]
smiles_df = pd.DataFrame(smiles, columns=["sm"+str(n) for n in range(1,bits+1)])
print("model_df: {:,} x {:,}".format(*model_df.shape))

#assert model_df.columns == smiles_df.columns

model_df = pd.concat([model_df,smiles_df],axis=1)
print("smiles_df: {:,} x {:,}".format(*smiles_df.shape))
print("model_df: {:,} x {:,}".format(*model_df.shape), "(added smiles features)")

del smiles, smiles_df

model_df: 638 x 91
smiles_df: 638 x 256
model_df: 638 x 347 (added smiles features)


Cell Type

In [210]:
#adding cell type feature

#need help here - not all cell types being added. for ex - rows 295 to 303

celltypes = []
for i in range(input_df.shape[0]):
    row = str(input_df.iloc[i].to_list())
    cell = re.findall('([a-zA-Z]+) cells', row) 
    if len(cell)==0:
        celltypes.append('Unknown')
    else:
        celltypes.append(cell[0])

print('Cell Types: ' + str(set(celltypes)))  
model_df['Cell Type'] = celltypes
print("\nmodel_df: {:,} x {:,}".format(*model_df.shape), "(added cell type feature)")

Cell Types: {'platelets', 'Kelly', 'germ', 'I', 'D', 'DB', 'MM', 'Mino', 'Unknown', 'hPBMC', 'PBMC', 'Jurkat', 'SR', 'VCaP', 'RKO', 't', 'HEL', 'HeLa', 'XLA', 'T', 'Ramos', 'NAMALWA', 'LNCaP', 'S'}

model_df: 638 x 348 (added cell type feature)


#### Response Variable (Binary)

In [206]:
# dfstr = input_df.select_dtypes(include=[object])
# rows = dfstr.apply(lambda x: x.str.contains("degradation",case=False)).any(axis=1)
# model_df.insert(0, "resp", rows.values.astype(int))

# print("model_df: {:,} x {:,}".format(*model_df.shape), "(added response variable)")
# del dfstr, rows

#### Response Variable (DC50)

In [ ]:
#split by cell type (need cell type to work first)

#### Model Matrices

In [ ]:
train,test = train_test_split(model_df, random_state=1, test_size=.2)

features = train.columns.tolist()[1:]  #dropping "resp"
print(f"nfeatures: {len(features):,}")

X_train = train.drop("resp", axis=1).values
y_train = train["resp"].values
print("X_train: {:,} x {:,}".format(*X_train.shape),  f"   %pos:{y_train.sum()/y_train.shape[0]:.2%}")

X_test = test.drop("resp", axis=1).values
y_test = test["resp"].values
print("X_test: {:,} x {:,}".format(*X_test.shape), f"   %pos:{y_test.sum()/y_test.shape[0]:.2%}")

del train, test

nfeatures: 557
X_train: 3,151 x 557    %pos:20.88%
X_test: 788 x 557    %pos:20.94%


In [ ]:
def eval_binary_classifier(clf):
  y_train_pred = clf.predict(X_train)
  y_test_pred = clf.predict(X_test)

  stats = [
    ["train",roc_auc_score(y_train,y_train_pred), average_precision_score(y_train,y_train_pred)],
    ["test",roc_auc_score(y_test,y_test_pred), average_precision_score(y_test,y_test_pred)]
  ]
  
  with pd.option_context("display.float_format", "{:,.2%}".format):
    print(pd.DataFrame(stats, columns=["","AUC","AP"]))

#### Model: RandomForest
```
min_samples_leaf  minimum number of samples required to be at a leaf node (incrementing to reduce overfitting)
```

In [ ]:
for msl in [10,5,2]:
  print(f"min_samples_leaf:{msl}")
  clf = RandomForestClassifier(random_state=10, n_estimators=50, min_samples_leaf=msl).fit(X_train,y_train)
  eval_binary_classifier(clf); print()
  
del clf, msl

min_samples_leaf:10
            AUC     AP
0  train 77.01% 61.70%
1   test 70.81% 51.65%

min_samples_leaf:5
            AUC     AP
0  train 84.71% 72.38%
1   test 77.08% 59.56%

min_samples_leaf:2
            AUC     AP
0  train 90.53% 81.75%
1   test 80.23% 62.37%

